# Lab 2: Building a Tool Registry System

**Duration:** 90-120 minutes  
**Level:** Advanced

## Learning Objectives

By the end of this lab, you will be able to:
1. Build a centralized tool registry
2. Implement type-safe tool schemas with Pydantic
3. Add authentication and authorization
4. Implement rate limiting and quotas
5. Version tools for backwards compatibility
6. Monitor tool usage with metrics

## Setup

In [ ]:
# Install required packages
!pip install openai pydantic -q

In [ ]:
import os
import json
import time
from typing import Dict, List, Any, Optional, Callable, Set
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from enum import Enum
from pydantic import BaseModel, Field, validator
from collections import defaultdict
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✓ Setup complete")

## Exercise 1: Basic Tool Registry

Build a centralized registry to manage all tools.

**Task:** Implement a ToolRegistry that stores tool metadata and provides lookup functionality.

In [ ]:
class ToolCategory(str, Enum):
    """Tool categories."""
    SEARCH = "search"
    DATA = "data"
    COMMUNICATION = "communication"
    COMPUTATION = "computation"

@dataclass
class ToolMetadata:
    """Metadata for a registered tool."""
    name: str
    description: str
    category: ToolCategory
    version: str
    parameters_schema: type[BaseModel]
    function: Callable
    enabled: bool = True
    tags: List[str] = field(default_factory=list)

# TODO: Implement ToolRegistry
class ToolRegistry:
    """Central registry for all tools."""
    
    def __init__(self):
        # TODO: Initialize storage for tools
        # TODO: Initialize call count tracking
        pass
    
    def register(
        self,
        name: str,
        description: str,
        category: ToolCategory,
        version: str,
        parameters_schema: type[BaseModel],
        function: Callable,
        **kwargs
    ) -> None:
        """Register a new tool."""
        # TODO: Check if tool already exists
        # TODO: Create ToolMetadata
        # TODO: Store in registry
        pass
    
    def get(self, name: str) -> Optional[ToolMetadata]:
        """Get tool by name."""
        # TODO: Return tool metadata
        pass
    
    def list_tools(
        self,
        category: Optional[ToolCategory] = None,
        enabled_only: bool = True
    ) -> List[ToolMetadata]:
        """List all tools with optional filtering."""
        # TODO: Get all tools
        # TODO: Filter by category if specified
        # TODO: Filter by enabled status
        pass
    
    def execute(self, name: str, **kwargs) -> Any:
        """Execute a tool."""
        # TODO: Get tool metadata
        # TODO: Validate it exists and is enabled
        # TODO: Validate parameters with Pydantic schema
        # TODO: Execute function
        # TODO: Track call count
        pass

# Test registry
registry = ToolRegistry()

# Define a simple tool
class CalculatorParams(BaseModel):
    a: float = Field(..., description="First number")
    b: float = Field(..., description="Second number")
    operation: str = Field(..., description="Operation: add, subtract, multiply, divide")

def calculator(a: float, b: float, operation: str) -> float:
    """Basic calculator."""
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        return a / b if b != 0 else float('inf')
    else:
        raise ValueError(f"Unknown operation: {operation}")

# Register tool
registry.register(
    name="calculator",
    description="Perform basic arithmetic operations",
    category=ToolCategory.COMPUTATION,
    version="1.0.0",
    parameters_schema=CalculatorParams,
    function=calculator,
    tags=["math", "basic"]
)

# Test execution
result = registry.execute("calculator", a=10, b=5, operation="add")
print(f"Result: {result}")

# List tools
tools = registry.list_tools()
print(f"Registered tools: {len(tools)}")

## Exercise 2: OpenAI Schema Conversion

Convert tool metadata to OpenAI function calling format.

**Task:** Implement method to convert Pydantic schemas to OpenAI tool schemas.

In [ ]:
# TODO: Add to ToolRegistry class
def to_openai_schema(registry: ToolRegistry, name: str) -> dict:
    """
    Convert tool to OpenAI function schema.
    
    Returns:
        OpenAI tool schema dict
    """
    # TODO: Get tool metadata
    # TODO: Extract Pydantic schema
    # TODO: Convert to OpenAI format
    pass

def get_all_openai_schemas(registry: ToolRegistry) -> List[dict]:
    """Get OpenAI schemas for all enabled tools."""
    # TODO: Get all enabled tools
    # TODO: Convert each to OpenAI schema
    pass

# Test conversion
schema = to_openai_schema(registry, "calculator")
print("OpenAI Schema:")
print(json.dumps(schema, indent=2))

# Use with OpenAI
all_schemas = get_all_openai_schemas(registry)
print(f"\nTotal schemas: {len(all_schemas)}")

## Exercise 3: Type-Safe Parameters with Pydantic

Create advanced Pydantic schemas with validation.

**Task:** Build tool parameter schemas with custom validators and constraints.

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field, validator

# TODO: Create WeatherParams with validation
class WeatherParams(BaseModel):
    """Weather query parameters with validation."""
    location: str = Field(..., min_length=2, max_length=100)
    unit: Literal["celsius", "fahrenheit", "kelvin"] = "celsius"
    include_forecast: bool = False
    
    # TODO: Add validator to capitalize location
    @validator('location')
    def validate_location(cls, v):
        # TODO: Check only letters and spaces
        # TODO: Capitalize words
        pass

# TODO: Create SearchParams with validation
class SearchParams(BaseModel):
    """Search parameters with validation."""
    query: str = Field(..., min_length=1, max_length=500)
    max_results: int = Field(default=10, ge=1, le=100)
    language: str = Field(default="en", regex="^[a-z]{2}$")
    
    # TODO: Add validator to clean query
    @validator('query')
    def clean_query(cls, v):
        # TODO: Strip whitespace
        # TODO: Remove multiple spaces
        pass

# Mock implementations
def get_weather(location: str, unit: str = "celsius", include_forecast: bool = False) -> dict:
    return {"location": location, "temp": 20, "unit": unit}

def search_web(query: str, max_results: int = 10, language: str = "en") -> dict:
    return {"query": query, "results": [f"Result {i}" for i in range(max_results)]}

# Register tools with validation
registry.register(
    name="get_weather",
    description="Get weather information",
    category=ToolCategory.DATA,
    version="1.0.0",
    parameters_schema=WeatherParams,
    function=get_weather
)

registry.register(
    name="search_web",
    description="Search the web",
    category=ToolCategory.SEARCH,
    version="1.0.0",
    parameters_schema=SearchParams,
    function=search_web
)

# Test validation
print("Testing validation...\n")

# Valid call
try:
    result = registry.execute("get_weather", location="london")
    print(f"✓ Valid: {result}")
except Exception as e:
    print(f"✗ Error: {e}")

# Invalid call (location too short)
try:
    result = registry.execute("get_weather", location="a")
    print(f"✓ Valid: {result}")
except Exception as e:
    print(f"✗ Error: {e}")

## Exercise 4: User Authentication & Authorization

Add role-based access control to tools.

**Task:** Implement user roles and check permissions before tool execution.

In [ ]:
class UserRole(str, Enum):
    ADMIN = "admin"
    USER = "user"
    GUEST = "guest"

@dataclass
class User:
    """User with roles."""
    id: str
    username: str
    roles: Set[UserRole]

# TODO: Implement AuthorizationManager
class AuthorizationManager:
    """Manage tool permissions."""
    
    def __init__(self):
        # TODO: Initialize tool permissions dict
        pass
    
    def set_tool_permissions(self, tool_name: str, roles: Set[UserRole]):
        """Set which roles can use a tool."""
        # TODO: Store permissions
        pass
    
    def can_use_tool(self, user: User, tool_name: str) -> bool:
        """Check if user can use tool."""
        # TODO: Get required roles
        # TODO: Check if user has any required role
        pass
    
    def authorize_or_raise(self, user: User, tool_name: str):
        """Raise exception if user cannot use tool."""
        # TODO: Check permission and raise if denied
        pass

# TODO: Extend registry with authorization
class SecureToolRegistry(ToolRegistry):
    """Registry with authorization."""
    
    def __init__(self):
        super().__init__()
        self.auth_manager = AuthorizationManager()
    
    def execute_as_user(self, user: User, tool_name: str, **kwargs) -> Any:
        """Execute tool with authorization check."""
        # TODO: Check authorization
        # TODO: Execute tool
        pass

# Test authorization
secure_registry = SecureToolRegistry()

# Register calculator (admin only)
secure_registry.register(
    name="calculator",
    description="Calculator",
    category=ToolCategory.COMPUTATION,
    version="1.0.0",
    parameters_schema=CalculatorParams,
    function=calculator
)

secure_registry.auth_manager.set_tool_permissions(
    "calculator",
    {UserRole.ADMIN, UserRole.USER}
)

# Test with different users
admin = User("1", "admin", {UserRole.ADMIN})
guest = User("2", "guest", {UserRole.GUEST})

print("Testing authorization...\n")

# Admin can use
try:
    result = secure_registry.execute_as_user(admin, "calculator", a=5, b=3, operation="add")
    print(f"✓ Admin: {result}")
except Exception as e:
    print(f"✗ Admin: {e}")

# Guest cannot use
try:
    result = secure_registry.execute_as_user(guest, "calculator", a=5, b=3, operation="add")
    print(f"✓ Guest: {result}")
except Exception as e:
    print(f"✗ Guest: {e}")

## Exercise 5: Rate Limiting & Quotas

Implement per-user rate limiting.

**Task:** Track tool usage and enforce rate limits.

In [ ]:
# TODO: Implement QuotaManager
class QuotaManager:
    """Manage per-user quotas."""
    
    def __init__(self):
        # TODO: Initialize call tracking
        # key: "user_id:tool_name" -> List[datetime]
        pass
    
    def record_call(self, user_id: str, tool_name: str):
        """Record a tool call."""
        # TODO: Record timestamp
        # TODO: Clean up old entries
        pass
    
    def get_usage(
        self,
        user_id: str,
        tool_name: str,
        window_hours: int = 24
    ) -> int:
        """Get usage count in time window."""
        # TODO: Count calls in window
        pass
    
    def check_quota(
        self,
        user_id: str,
        tool_name: str,
        limit: int,
        window_hours: int = 24
    ) -> bool:
        """Check if under quota."""
        # TODO: Check usage vs limit
        pass
    
    def enforce_quota(
        self,
        user_id: str,
        tool_name: str,
        limit: int,
        window_hours: int = 24
    ):
        """Raise exception if quota exceeded."""
        # TODO: Check quota and raise if exceeded
        pass

# TODO: Add quota enforcement to registry
class RateLimitedRegistry(SecureToolRegistry):
    """Registry with rate limiting."""
    
    def __init__(self):
        super().__init__()
        self.quota_manager = QuotaManager()
    
    def execute_as_user(
        self,
        user: User,
        tool_name: str,
        quota_limit: int = 100,
        **kwargs
    ) -> Any:
        """Execute with quota enforcement."""
        # TODO: Check authorization
        # TODO: Enforce quota
        # TODO: Execute tool
        # TODO: Record call
        pass

# Test rate limiting
limited_registry = RateLimitedRegistry()

limited_registry.register(
    name="calculator",
    description="Calculator",
    category=ToolCategory.COMPUTATION,
    version="1.0.0",
    parameters_schema=CalculatorParams,
    function=calculator
)

limited_registry.auth_manager.set_tool_permissions(
    "calculator",
    {UserRole.ADMIN, UserRole.USER}
)

user = User("1", "testuser", {UserRole.USER})

print("Testing rate limiting...\n")

# Make multiple calls
for i in range(12):
    try:
        result = limited_registry.execute_as_user(
            user,
            "calculator",
            quota_limit=10,
            a=i,
            b=1,
            operation="add"
        )
        print(f"✓ Call {i+1}: {result}")
    except Exception as e:
        print(f"✗ Call {i+1}: {e}")

## Exercise 6: Tool Versioning

Support multiple versions of the same tool.

**Task:** Build a versioned registry that can run different tool versions.

In [ ]:
# TODO: Implement VersionedToolRegistry
class VersionedToolRegistry:
    """Registry supporting multiple tool versions."""
    
    def __init__(self):
        # TODO: tool_name -> version -> metadata
        # TODO: default_versions dict
        pass
    
    def register_version(
        self,
        name: str,
        version: str,
        metadata: ToolMetadata,
        set_as_default: bool = False
    ):
        """Register a specific version."""
        # TODO: Store versioned metadata
        # TODO: Update default version if needed
        pass
    
    def get_version(
        self,
        name: str,
        version: Optional[str] = None
    ) -> Optional[ToolMetadata]:
        """Get specific version or default."""
        # TODO: Return specified version or default
        pass
    
    def list_versions(self, name: str) -> List[str]:
        """List all versions of a tool."""
        # TODO: Return sorted list of versions
        pass
    
    def execute(
        self,
        name: str,
        version: Optional[str] = None,
        **kwargs
    ) -> Any:
        """Execute specific version."""
        # TODO: Get version
        # TODO: Validate parameters
        # TODO: Execute
        pass

# Test versioning
versioned_registry = VersionedToolRegistry()

# Version 1.0.0: Simple calculator
class CalcV1Params(BaseModel):
    a: float
    b: float

def calc_v1(a: float, b: float) -> float:
    """V1: Only addition"""
    return a + b

versioned_registry.register_version(
    name="calculator",
    version="1.0.0",
    metadata=ToolMetadata(
        name="calculator",
        description="Add two numbers",
        category=ToolCategory.COMPUTATION,
        version="1.0.0",
        parameters_schema=CalcV1Params,
        function=calc_v1
    )
)

# Version 2.0.0: With operations
versioned_registry.register_version(
    name="calculator",
    version="2.0.0",
    metadata=ToolMetadata(
        name="calculator",
        description="Calculator with operations",
        category=ToolCategory.COMPUTATION,
        version="2.0.0",
        parameters_schema=CalculatorParams,
        function=calculator
    ),
    set_as_default=True
)

print("Testing versioning...\n")

# Use default version (2.0.0)
result = versioned_registry.execute("calculator", a=10, b=5, operation="multiply")
print(f"Default (v2.0.0): {result}")

# Use specific version (1.0.0)
result = versioned_registry.execute("calculator", version="1.0.0", a=10, b=5)
print(f"v1.0.0: {result}")

# List versions
versions = versioned_registry.list_versions("calculator")
print(f"\nAvailable versions: {versions}")

## Exercise 7: Usage Monitoring

Track and analyze tool usage patterns.

**Task:** Build a monitoring system that provides insights into tool usage.

In [ ]:
@dataclass
class ToolUsageMetric:
    """Single tool usage record."""
    tool_name: str
    user_id: str
    timestamp: datetime
    duration_ms: float
    success: bool
    error: Optional[str] = None

# TODO: Implement UsageMonitor
class UsageMonitor:
    """Monitor tool usage."""
    
    def __init__(self):
        # TODO: Initialize metrics storage
        pass
    
    def record(self, metric: ToolUsageMetric):
        """Record a usage metric."""
        # TODO: Store metric
        pass
    
    def get_tool_stats(self, tool_name: str) -> Dict:
        """Get statistics for a tool."""
        # TODO: Filter metrics for tool
        # TODO: Calculate total calls, success rate
        # TODO: Calculate average latency, p95
        pass
    
    def get_user_stats(self, user_id: str) -> Dict:
        """Get statistics for a user."""
        # TODO: Filter metrics for user
        # TODO: Calculate total calls by tool
        # TODO: Calculate most used tools
        pass
    
    def get_overall_stats(self) -> Dict:
        """Get overall statistics."""
        # TODO: Calculate total calls
        # TODO: Calculate success rate
        # TODO: Most popular tools
        # TODO: Active users
        pass

# TODO: Add monitoring to registry
class MonitoredRegistry(RateLimitedRegistry):
    """Registry with usage monitoring."""
    
    def __init__(self):
        super().__init__()
        self.monitor = UsageMonitor()
    
    def execute_as_user(
        self,
        user: User,
        tool_name: str,
        quota_limit: int = 100,
        **kwargs
    ) -> Any:
        """Execute with monitoring."""
        start = time.time()
        
        try:
            # TODO: Execute tool (use parent class)
            result = None
            
            # TODO: Record success metric
            
            return result
            
        except Exception as e:
            # TODO: Record failure metric
            raise

# Test monitoring
monitored_registry = MonitoredRegistry()

monitored_registry.register(
    name="calculator",
    description="Calculator",
    category=ToolCategory.COMPUTATION,
    version="1.0.0",
    parameters_schema=CalculatorParams,
    function=calculator
)

monitored_registry.auth_manager.set_tool_permissions(
    "calculator",
    {UserRole.ADMIN, UserRole.USER}
)

# Simulate usage
import random

users = [
    User("1", "alice", {UserRole.USER}),
    User("2", "bob", {UserRole.USER}),
    User("3", "charlie", {UserRole.ADMIN})
]

for i in range(50):
    user = random.choice(users)
    try:
        monitored_registry.execute_as_user(
            user,
            "calculator",
            quota_limit=100,
            a=random.randint(1, 10),
            b=random.randint(1, 10),
            operation=random.choice(["add", "subtract", "multiply"])
        )
    except:
        pass

# Get statistics
print("\n=== Usage Statistics ===")

tool_stats = monitored_registry.monitor.get_tool_stats("calculator")
print("\nTool Stats:")
print(json.dumps(tool_stats, indent=2))

overall = monitored_registry.monitor.get_overall_stats()
print("\nOverall Stats:")
print(json.dumps(overall, indent=2))

## Exercise 8: Complete Registry Integration

Use the registry with OpenAI for real conversations.

**Task:** Build an agent that uses your registry with all features enabled.

In [ ]:
class RegistryAgent:
    """Agent using tool registry."""
    
    def __init__(self, registry: MonitoredRegistry, user: User):
        self.registry = registry
        self.user = user
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.conversation = []
    
    def chat(self, message: str, quota_limit: int = 100) -> str:
        """Chat with tool access."""
        # Add user message
        self.conversation.append({"role": "user", "content": message})
        
        # Get tool schemas
        tools = get_all_openai_schemas(self.registry)
        
        # Initial API call
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant with access to tools."},
                *self.conversation
            ],
            tools=tools
        )
        
        msg = response.choices[0].message
        
        # Handle tool calls
        if msg.tool_calls:
            self.conversation.append(msg)
            
            for tool_call in msg.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"🔧 Calling {function_name}({function_args})")
                
                # Execute through registry
                try:
                    result = self.registry.execute_as_user(
                        self.user,
                        function_name,
                        quota_limit=quota_limit,
                        **function_args
                    )
                    tool_result = {"success": True, "result": result}
                except Exception as e:
                    tool_result = {"success": False, "error": str(e)}
                
                self.conversation.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(tool_result)
                })
            
            # Second API call
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant with access to tools."},
                    *self.conversation
                ],
                tools=tools
            )
            
            msg = response.choices[0].message
        
        self.conversation.append(msg)
        return msg.content

# Test agent
user = User("1", "testuser", {UserRole.USER})
agent = RegistryAgent(monitored_registry, user)

response = agent.chat("What is 15 + 27?")
print(f"\nAgent: {response}")

# Check stats
stats = monitored_registry.monitor.get_user_stats(user.id)
print(f"\nUser stats: {json.dumps(stats, indent=2)}")

## Summary

### Key Concepts Covered

1. **Tool Registry**: Centralized management of all tools
2. **Type Safety**: Pydantic schemas for parameter validation
3. **Authorization**: Role-based access control
4. **Rate Limiting**: Per-user quotas and usage tracking
5. **Versioning**: Multiple versions of the same tool
6. **Monitoring**: Usage statistics and analytics
7. **Integration**: Complete OpenAI integration

### Production Checklist

- [ ] Implement centralized tool registry
- [ ] Add Pydantic schemas for all tools
- [ ] Configure role-based permissions
- [ ] Set up per-user rate limiting
- [ ] Version all tools semantically
- [ ] Enable usage monitoring
- [ ] Convert to OpenAI schemas automatically
- [ ] Test authorization edge cases
- [ ] Monitor quota usage
- [ ] Document tool APIs

### Next Steps

- Lab 3: Multi-Tool Workflow Orchestration
- Lesson 3: Tool Orchestration & Workflows